In [2]:
import numpy as np 
import os
import pandas as pd
import matplotlib
import cv2

In [3]:
DATA_PATH="/kaggle/input/covidlarge/train"
CATEGORIES=['Covid', 'NORMAL', 'PNEUMONIA']
img_size=150
training_data = []
def create_training_data():
    #iterating through different categories
    for category in CATEGORIES:
        path=os.path.join(DATA_PATH, category)
        print(path)
        class_num = CATEGORIES.index(category)
        print(class_num)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array,(img_size,img_size))
                training_data.append([new_array,class_num])
            except Exception as e:
                pass

In [4]:
create_training_data()
print(len(training_data))

/kaggle/input/covidlarge/train/Covid
0
/kaggle/input/covidlarge/train/NORMAL
1
/kaggle/input/covidlarge/train/PNEUMONIA
2
18707


In [5]:
import random
random.shuffle(training_data)

In [6]:
X=[]
y=[]

for features,label in training_data: 
	X.append(features)
	y.append(label)


X=np.array(X).reshape(-1, img_size, img_size, 1) 
print(X.shape)
print(len(y))

(18707, 150, 150, 1)
18707


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LeakyReLU
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [8]:
X=X/255
model=Sequential()
model.add(Conv2D(32, (3,3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
#model.add(Conv2D(32,(3,3)))
#model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))



#model.add(Conv2D(64, (3,3)))
#model.add(Activation('relu'))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))

#model.add(Conv2D(256, (3,3)))
#model.add(Activation('relu'))
model.add(Conv2D(256, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))

#model.add(Conv2D(512, (3,3)))
#model.add(Activation('relu'))
model.add(Conv2D(512, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(1024, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))

#model.add(Conv2D(2048, (3,3)))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(2,2))




model.add(Flatten())
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(300))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(3))
model.add(Activation('softmax'))
model.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      320       
_________________________________________________________________
activation (Activation)      (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 72, 72, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 256)       1

In [9]:
target=to_categorical(y, num_classes=3)

In [10]:
batch_size=64
epochs=15

checkpoint = ModelCheckpoint(filepath='covid19.h5', save_best_only=True)
#lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=4, verbose=2, mode='max')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=1, mode='min')

In [11]:
model.compile(optimizer='Adamax', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
history=model.fit(X, target, epochs=epochs, batch_size=batch_size, validation_split=0.2, callbacks=[checkpoint])

Train on 14965 samples, validate on 3742 samples
Epoch 1/15
14965/14965 [==============================] - 17s 1ms/sample - loss: 0.6235 - accuracy: 0.6511 - val_loss: 0.4296 - val_accuracy: 0.8215
Epoch 2/15
14965/14965 [==============================] - 12s 801us/sample - loss: 0.3922 - accuracy: 0.8483 - val_loss: 0.3508 - val_accuracy: 0.8691
Epoch 3/15
14965/14965 [==============================] - 12s 795us/sample - loss: 0.3421 - accuracy: 0.8755 - val_loss: 0.3203 - val_accuracy: 0.8776
Epoch 4/15
14965/14965 [==============================] - 12s 791us/sample - loss: 0.3078 - accuracy: 0.8850 - val_loss: 0.2768 - val_accuracy: 0.8958
Epoch 5/15
14965/14965 [==============================] - 12s 793us/sample - loss: 0.2824 - accuracy: 0.8982 - val_loss: 0.2722 - val_accuracy: 0.9006
Epoch 6/15
14965/14965 [==============================] - 12s 807us/sample - loss: 0.2620 - accuracy: 0.9065 - val_loss: 0.2681 - val_accuracy: 0.8939
Epoch 7/15
14965/14965 [=======================

In [13]:
model.save('covidLarge.h5')